<a href="https://colab.research.google.com/github/cristianopoeta/DSWP_aluno/blob/main/Notebooks/NB10_04__Transformation_exerc_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercícios 6 - 120 years of Olympic history: athletes and results
* [120 years of Olympic history: athletes and results](https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results)
    * Trate adequadamente as variáveis 'sex', 'season', 'team', 'city', 'sport' e 'medal';
    * Aplique as transformações que acabamos de estudar nos campos/colunas numéricas 'height' e 'weight'. Cuidado com os Missing Values contidos nas variáveis!
    * Verifique/avalie o impacto dos outliers nestas colunas.
    * Neste caso, qual transformação é mais adequado diante dos outliers?

### Definições iniciais

In [35]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Markdown

In [36]:
pdod = pd.options.display    # atalho para opções de exibição
pdod.max_rows = 100          # qtd máx de linhas exibidas
pdod.max_columns = 100       # qtd máx de colunas exibidas
pdod.width = 200             # larg máx total em modo texto
d = display    # atalho para `display`

In [37]:
# retorna para cada coluna de `frame`: nome da coluna, valor de exemplo, 
#     dtype da coluna, classe do valor de exemplo
# valor de exemplo é tomado na linha de `frame` indicada por `iloc`
def exemplo_linha(frame, iloc=0):
  df_info = pd.DataFrame(dict(valor_exemplo=frame.iloc[iloc].copy()))
  df_info['dtype_coluna'] = frame.dtypes.map(lambda x: x.name)
  df_info['classe_valor'] = df_info['valor_exemplo'].map(lambda x: x.__class__.__name__)
  df_info.index.name = 'nome_coluna'
  return df_info

In [38]:
# `obj` pode ser DataFrame ou Series.
# se `filtros` for `None`, exibe primeiras `nh` posições, últimas `nt` posições 
#     e `shape` de `obj`.
# se `filtros` não for `None`, deve ser um iterável com elemntos que possam ser 
#     utilizados em `obj.loc[]` (funções de filtragem são uma boa opção).
def d_pd(obj, nh=1, nt=None, filtros=None):
    if nt is None:
        nt = nh
    if filtros is None:
        display(pd.concat([obj.head(nh), obj.tail(nt)]), obj.shape)
    else:
        for filtro in filtros:
            display(obj.loc[filtro])
        display(obj.shape)

In [39]:
# monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importação do dataset e verificações e modificações iniciais

In [40]:
# carrega dataset
df_olimp = pd.read_csv('/content/drive/My Drive/DSWP/athlete_events.csv')
d_pd(df_olimp)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
271115,135571,Tomasz Ireneusz ya,M,34.0,185.0,96.0,Poland,POL,2002 Winter,2002,Winter,Salt Lake City,Bobsleigh,Bobsleigh Men's Four,NaN


(271116, 15)

In [41]:
# conversão dos nomes de colunas para minúsculas
df_olimp.set_axis(df_olimp.columns.str.lower(), axis=1, inplace=True)

In [42]:
# inspeção de valores e tipos de dados
exemplo_linha(df_olimp).T

nome_coluna,id,name,sex,age,height,weight,team,noc,games,year,season,city,sport,event,medal
valor_exemplo,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
dtype_coluna,int64,object,object,float64,float64,float64,object,object,object,int64,object,object,object,object,object
classe_valor,int64,str,str,float64,float64,float64,str,str,str,int64,str,str,str,str,float


### Trate adequadamente as variáveis 'sex', 'season', 'team', 'city', 'sport' e 'medal';


##### Com `pandas`

In [75]:
# inspeção das variáveis categóricas
df_categs = df_olimp[['sex', 'season', 'team', 'city', 'sport', 'medal']]
d_pd(df_categs)
print()
d(df_categs.describe())
print()
d(exemplo_linha(df_categs.sample()).T)

,sex,season,team,city,sport,medal
0,M,Summer,China,Barcelona,Basketball,NaN
271115,M,Winter,Poland,Salt Lake City,Bobsleigh,NaN


(271116, 6)

,sex,season,team,city,sport,medal
count,271116,271116,271116,271116,271116,39783
unique,2,2,1184,42,66,3
top,M,Summer,United States,London,Athletics,Gold
freq,196594,222552,17847,22426,38624,13372


nome_coluna,sex,season,team,city,sport,medal
valor_exemplo,M,Summer,Uruguay,Roma,Cycling,NaN
dtype_coluna,object,object,object,object,object,object
classe_valor,str,str,str,str,str,float


In [77]:
# domínios das colunas 'sex', 'season', 'medal'.
for col in ['sex', 'season', 'medal']:
  d(df_categs[col].value_counts(dropna=False).to_frame().T)
  print()

,M,F
sex,196594,74522


,Summer,Winter
season,222552,48564


,NaN,Gold,Bronze,Silver
medal,231333,13372,13295,13116


In [83]:
# `pandas.get`
df_dumm_1 = pd.get_dummies(df_categs[['sex', 'season']])
d(df_dumm_1.sample(5).join(df_categs))
print()
df_dumm_2 = pd.get_dummies(df_categs[['sex', 'season']], drop_first=True)
d(df_dumm_2.sample(5).join(df_categs))

,sex_F,sex_M,season_Summer,season_Winter,sex,season,team,city,sport,medal
19268,1,0,1,0,F,Summer,Italy,Beijing,Gymnastics,NaN
224286,0,1,1,0,M,Summer,Czechoslovakia,Berlin,Athletics,NaN
71202,0,1,1,0,M,Summer,Canada,Atlanta,Rowing,NaN
247228,1,0,0,1,F,Winter,Estonia,Nagano,Cross Country Skiing,NaN
45964,0,1,1,0,M,Summer,Australia,Melbourne,Athletics,NaN


,sex_M,season_Winter,sex,season,team,city,sport,medal
58940,1,0,M,Summer,Great Britain,Moskva,Weightlifting,NaN
265077,1,0,M,Summer,Soviet Union,Moskva,Cycling,Gold
188008,0,0,F,Summer,Netherlands,Los Angeles,Swimming,NaN
188124,1,0,M,Summer,Fiji,Barcelona,Sailing,NaN
263542,1,0,M,Summer,Switzerland,Los Angeles,Gymnastics,NaN


In [86]:
pdod.max_rows = None
df_categs.team.value_counts().sort_index()

30. Februar                                            2
A North American Team                                  4
Acipactli                                              3
Acturus                                                2
Afghanistan                                          126
Akatonbo                                               3
Alain IV                                               3
Albania                                               70
Alcaid                                                 3
Alcyon-6                                               1
Alcyon-7                                               2
Aldebaran                                              2
Aldebaran II                                           2
Aletta                                                 3
Algeria                                              551
Ali-Baba II                                            7
Ali-Baba IV                                            2
Ali-Baba IX                    

### Aplique as transformações que acabamos de estudar nos campos/colunas numéricas 'height' e 'weight'. Cuidado com os Missing Values contidos nas variáveis!


### Verifique/avalie o impacto dos outliers nestas colunas.


### Neste caso, qual transformação é mais adequado diante dos outliers?

### Rascunho

In [43]:
# criação de um dataframe somente com as variáveis do exercício
df_vars = df_olimp[[ 'sex', 'season', 'team', 'city', 'sport', 'medal']]
d_pd(df_vars, 3)
print()
exemplo_linha(df_vars).T

,sex,season,team,city,sport,medal
0,M,Summer,China,Barcelona,Basketball,NaN
1,M,Summer,China,London,Judo,NaN
2,M,Summer,Denmark,Antwerpen,Football,NaN
271113,M,Winter,Poland,Sochi,Ski Jumping,NaN
271114,M,Winter,Poland,Nagano,Bobsleigh,NaN
271115,M,Winter,Poland,Salt Lake City,Bobsleigh,NaN


(271116, 6)

nome_coluna,sex,season,team,city,sport,medal
valor_exemplo,M,Summer,China,Barcelona,Basketball,NaN
dtype_coluna,object,object,object,object,object,object
classe_valor,str,str,str,str,str,float


In [44]:
df_vars.describe()

,sex,season,team,city,sport,medal
count,271116,271116,271116,271116,271116,39783
unique,2,2,1184,42,66,3
top,M,Summer,United States,London,Athletics,Gold
freq,196594,222552,17847,22426,38624,13372


In [46]:
pd.get_dummies(df_vars[['sex', 'season']], drop_first=True)

,sex_M,season_Winter
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
271111,1,1
271112,1,1
271113,1,1
271114,1,1


In [47]:
df_vars.medal.map({np.nan:0, 'Bronze':1, 'Silver':2, 'Gold':4})

0         0
1         0
2         0
3         4
4         0
         ..
271111    0
271112    0
271113    0
271114    0
271115    0
Name: medal, Length: 271116, dtype: int64

In [48]:
from sklearn.preprocessing import (
    LabelEncoder, OneHotEncoder, OrdinalEncoder, 
    StandardScaler, RobustScaler, MaxAbsScaler, MinMaxScaler)

In [49]:
lbl_e = OrdinalEncoder()

In [51]:
lbl_e?

In [52]:
lbl_e.fit_transform(df_olimp[['sex']])

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [54]:
lbl_e.categories_

[array(['F', 'M'], dtype=object)]

In [55]:
ohe = OneHotEncoder()

In [59]:
ohe 

In [57]:
ohe.fit_transform(df_olimp[['sex', 'season']])

<271116x2 sparse matrix of type '<class 'numpy.float64'>'
	with 271116 stored elements in Compressed Sparse Row format>

### Exemplo de transformação do dataframe de correlações em uma série de correlações


In [60]:
# criação de um dataframe de correlações de exemplo
df_corr = (
    df_olimp
    .select_dtypes('number')
    .drop(columns='id')
    .corr())
df_corr

,age,height,weight,year
age,1.000000,0.138246,0.212069,-0.115137
height,0.138246,1.000000,0.796213,0.047578
weight,0.212069,0.796213,1.000000,0.019095
year,-0.115137,0.047578,0.019095,1.000000


##### Com *method chaining*

In [64]:
# com encadeamento de métodos (method chaining)
(
    df_corr
    .pipe(lambda df: pd.DataFrame(
        np.where(np.triu(np.ones(df.shape, dtype=int), k=1), df, np.nan),
        index=df.index, columns=df.columns))
    .stack()
    .dropna()
    .sort_values(ascending=False, key=abs)
)

height  weight    0.796213
age     weight    0.212069
        height    0.138246
        year     -0.115137
height  year      0.047578
weight  year      0.019095
dtype: float64

##### Passo a passo

In [69]:
# máscara para selecionar somente correlações do 
# triângulo superior de `df_corr`.
# tipo `int` não é necessário, mas foi escolhido
# para tornar `mask_triu` mais legível
mask_triu = np.triu(np.ones(df_corr.shape, dtype=int), k=1)
mask_triu

array([[0, 1, 1, 1],
       [0, 0, 1, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 0]])

In [70]:
# aplicação da máscara em `df_corr` com `numpy.where()`:
# - na posição de `mask_triu` com 1 (True), o valor vem 
#     de `df_corr`;
# - na posição de `mask_triu` com 0 (True), o valor é NaN.
arr_corr_mskd = np.where(mask_triu, df_corr, np.nan)
arr_corr_mskd

array([[        nan,  0.13824647,  0.21206886, -0.11513685],
       [        nan,         nan,  0.79621309,  0.0475779 ],
       [        nan,         nan,         nan,  0.0190946 ],
       [        nan,         nan,         nan,         nan]])

In [71]:
# versão de `df_corr` com NaN na diagonal e 
# no triângulo inferior
df_corr_mskd = pd.DataFrame(data=arr_corr_mskd, 
    index=df_corr.index, columns=df_corr.columns
)
df_corr_mskd

,age,height,weight,year
age,NaN,0.138246,0.212069,-0.115137
height,NaN,NaN,0.796213,0.047578
weight,NaN,NaN,NaN,0.019095
year,NaN,NaN,NaN,NaN


In [72]:
# series com as coorelações em ordem 
# decrescente de valor absoluto
( df_corr_mskd
  .stack()
  .dropna()
  .sort_values(ascending=False, key=abs) )

height  weight    0.796213
age     weight    0.212069
        height    0.138246
        year     -0.115137
height  year      0.047578
weight  year      0.019095
dtype: float64